# Pre processing

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import pickle as pk
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten,Reshape, Input, Add
from tensorflow.keras.layers import Conv2D, MaxPooling2D,UpSampling2D,InputLayer,ReLU,BatchNormalization
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from scipy.spatial import distance_matrix
import plotly.express as px
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVD.csv')

In [6]:
with open('/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Audio_Embeddings/VGGish/embeddings_average_VGGish.pkl', "rb") as fp:
    dict_emb_VGG_av = pk.load(fp)
with open('/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Audio_Embeddings/VGGish/embeddings_max_pooling_VGGish.pkl', "rb") as fp:
    dict_emb_VGG_mp = pk.load(fp)
with open('/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Video_Embeddings/all_embeddings_sentenceBERT.pkl', "rb") as fp:
    dict_emb_yolo_sent = pk.load(fp)

In [7]:
emb_MPnet = np.load('/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Texte_Embeddings/MPnet.npy')
dict_emb_MPnet = {}
for count, name in enumerate(list(df['name'])):
  dict_emb_MPnet[name] = emb_MPnet[count]

In [8]:
dict_emb_yolo_sent[df['name'][0]].shape

(768,)

In [9]:
df_test = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Test_SurSampling1.csv")
df_train = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Train_SurSampling1.csv")
df_valid = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Val_SurSampling1.csv")
len(df_test), len(df_train), len(df_valid)

(640, 1909, 639)

In [10]:
len(set(df_test['name']))

456

In [ ]:
emb_VGG_av_train = np.zeros((len(df_train), 256))
emb_yolo_sent_train = np.zeros((len(df_train), 768))
emb_MPnet_train = np.zeros((len(df_train), 768))
for count, name in enumerate(df_train['name']):
  emb_VGG_av_train[count] = np.concatenate((dict_emb_VGG_av[name], dict_emb_VGG_mp[name]), axis = 0)
  emb_yolo_sent_train[count] = dict_emb_yolo_sent[name]
  emb_MPnet_train[count] = dict_emb_MPnet[name]

emb_VGG_av_test = np.zeros((len(df_test), 256))
emb_yolo_sent_test = np.zeros((len(df_test), 768))
emb_MPnet_test = np.zeros((len(df_test), 768))
for count, name in enumerate(df_test['name']):
  emb_VGG_av_test[count] = np.concatenate((dict_emb_VGG_av[name], dict_emb_VGG_mp[name]), axis = 0)
  emb_yolo_sent_test[count] = dict_emb_yolo_sent[name]
  emb_MPnet_test[count] = dict_emb_MPnet[name]

emb_VGG_av_valid = np.zeros((len(df_valid), 256))
emb_yolo_sent_valid = np.zeros((len(df_valid), 768))
emb_MPnet_valid = np.zeros((len(df_valid), 768))
for count, name in enumerate(df_valid['name']):
  emb_VGG_av_valid[count] = np.concatenate((dict_emb_VGG_av[name], dict_emb_VGG_mp[name]), axis = 0)
  emb_yolo_sent_valid[count] = dict_emb_yolo_sent[name]
  emb_MPnet_valid[count] = dict_emb_MPnet[name]

In [ ]:
def nearest(pred_space, ind, Minsk = 1):
  dist = distance_matrix(pred_space, ind, Minsk)
  #dist += np.diag([10**3]*len(dist))
  pred=np.zeros(len(dist))
  for i in range(len(dist)):
    pred[i] = np.argmin(dist[i])
  return pred

In [ ]:
def eval_pred(pred):
  s = 0
  for count, elem in enumerate(pred):
    if count == elem:
      s+=1
  return s/len(pred)

In [ ]:
from sklearn.metrics import confusion_matrix
def show_conf_matrix(y_test, pred):
  list_cat= list(set(y_test))
  conf_matrix = confusion_matrix(y_test, pred, labels=list_cat)
  cf = [[i/sum(ligne) for i in ligne] for ligne in conf_matrix]
  data = cf
  fig = px.imshow(data,
                  labels=dict(y="Vrai Catégorie", x="Catégorie prédite", color="Productivity"),
                  x=list_cat,
                  y=list_cat
                )
  fig.update_xaxes(side="top")
  fig.show()

# Model d'une à l'autre

In [ ]:
def getAutoEncoder_256_to_768(encodim_dim = 2, loss = 'mae'):
  x_in = Input((256))
  x = x_in
  layers=[512, 512, 512]
  for layer in layers:
    x = Dense(layer, activation = None)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.25)(x)
  features = Dense (encodim_dim, activation='relu', name='featurespace')(x)
  print(features)
  x = features
  for layer in layers:
    x = Dense(layer, activation = None)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.25)(x)
  x_out = Dense(768, activation = 'ReLU')(x)
  model = Model(x_in, x_out)
  model_features = Model(x_in, features)
  model.compile(loss = loss, optimizer = Adam())
  return model, model_features

model_end_256_to_768, model_features = getAutoEncoder_256_to_768(encodim_dim = 32)
#model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='featurespace/Relu:0', description="created by layer 'featurespace'")


In [ ]:
def Direct_256_to_768(loss = 'mae', dropout = True):
  x_in = Input((256))
  x = x_in
  x = BatchNormalization()(x)
  layers=[ 512, 512, 512, 512, 1024]
  for layer in layers:
    x = Dense(layer, activation = None)(x)
    x = ReLU()(x)
    if dropout:
      x = Dropout(0.25)(x)
  x_out = Dense(768, activation = 'ReLU')(x)
  model = Model(x_in, x_out)
  model.compile(loss = loss, optimizer = Adam())
  return model

model_direct_256_to_768= Direct_256_to_768()
#model.summary()

In [ ]:
def getAutoEncoder_768_to_256(encodim_dim = 2, loss = 'mae'):
  x_in = Input((768))
  x = x_in
  x = BatchNormalization()(x)
  layers=[512, 512, 512]
  for layer in layers:
    x = Dense(layer, activation = None)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.25)(x)
  features = Dense (encodim_dim, activation='ReLU', name='featurespace')(x)
  print(features)
  x = features
  for layer in layers:
    x = Dense(layer, activation = None)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.25)(x)
  x_out = Dense(256, activation = 'relu')(x)
  model = Model(x_in, x_out)
  model_features = Model(x_in, features)
  model.compile(loss = loss, optimizer = Adam())
  return model, model_features

model_end_768_to_256, model_features = getAutoEncoder_256_to_768(encodim_dim = 32)
#model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='featurespace/Relu:0', description="created by layer 'featurespace'")


In [ ]:
def Direct_768_to_256(loss = 'mae', dropout = True):
  x_in = Input((768))
  x = x_in
  x = BatchNormalization()(x)
  layers=[1024, 512, 512]
  for layer in layers:
    x = Dense(layer, activation = None)(x)
    x = ReLU()(x)
    if dropout:
      x = Dropout(0.25)(x)
    #x = Dropout(0.5)(x)
  x_out = Dense(256, activation = 'ReLU')(x)
  model = Model(x_in, x_out)
  model.compile(loss = loss, optimizer = Adam())
  return model

model_direct_768_to_256= Direct_256_to_768()
#model.summary()

# Entraînement

## Direct

### 256 -> 768

In [ ]:
model_direct_256_to_768= Direct_256_to_768(dropout = False)
batch_size = 64
epochs=50
hist = model_direct_256_to_768.fit(emb_VGG_av_train, emb_MPnet_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(emb_VGG_av_valid, emb_MPnet_valid))
model_direct_256_to_768.evaluate(emb_VGG_av_test, emb_MPnet_test)

Epoch 1/50
30/30 [==============================] - 7s 12ms/step - loss: 0.0294 - val_loss: 0.0278
Epoch 2/50
30/30 [==============================] - 0s 7ms/step - loss: 0.0275 - val_loss: 0.0277
Epoch 3/50
30/30 [==============================] - 0s 6ms/step - loss: 0.0273 - val_loss: 0.0277
Epoch 4/50
30/30 [==============================] - 0s 9ms/step - loss: 0.0271 - val_loss: 0.0276
Epoch 5/50
30/30 [==============================] - 0s 10ms/step - loss: 0.0269 - val_loss: 0.0275
Epoch 6/50
30/30 [==============================] - 0s 8ms/step - loss: 0.0267 - val_loss: 0.0274
Epoch 7/50
30/30 [==============================] - 0s 10ms/step - loss: 0.0266 - val_loss: 0.0273
Epoch 8/50
30/30 [==============================] - 0s 9ms/step - loss: 0.0263 - val_loss: 0.0272
Epoch 9/50
30/30 [==============================] - 0s 10ms/step - loss: 0.0261 - val_loss: 0.0272
Epoch 10/50
30/30 [==============================] - 0s 10ms/step - loss: 0.0259 - val_loss: 0.0271
Epoch 11/50
30

0.027075612917542458

In [ ]:
pred = model_direct_256_to_768.predict(emb_VGG_av_test)
len(pred)

20/20 [==============================] - 0s 2ms/step


640

In [ ]:
pred = model_direct_256_to_768.predict(emb_VGG_av_test)
n1 = nearest(pred, emb_MPnet_test, Minsk = 1)
n2 = nearest(pred, emb_MPnet_test, Minsk = 2)
print(len(set(n1)) , len(set(n2)))
print(eval_pred(n1), eval_pred(n2))

20/20 [==============================] - 0s 2ms/step
104 110
0.0421875 0.04375


In [ ]:
model_direct_256_to_768= Direct_256_to_768()
batch_size = 64
epochs=30
hist = model_direct_256_to_768.fit(emb_VGG_av_train, emb_yolo_sent_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(emb_VGG_av_valid, emb_yolo_sent_valid))
model_direct_256_to_768.evaluate(emb_VGG_av_test, emb_yolo_sent_test)

Epoch 1/30
30/30 [==============================] - 3s 13ms/step - loss: 0.1914 - val_loss: 0.1912
Epoch 2/30
30/30 [==============================] - 0s 7ms/step - loss: 0.1848 - val_loss: 0.1869
Epoch 3/30
30/30 [==============================] - 0s 7ms/step - loss: 0.1833 - val_loss: 0.1864
Epoch 4/30
30/30 [==============================] - 0s 9ms/step - loss: 0.1822 - val_loss: 0.1848
Epoch 5/30
30/30 [==============================] - 0s 7ms/step - loss: 0.1814 - val_loss: 0.1836
Epoch 6/30
30/30 [==============================] - 0s 8ms/step - loss: 0.1805 - val_loss: 0.1829
Epoch 7/30
30/30 [==============================] - 0s 7ms/step - loss: 0.1795 - val_loss: 0.1828
Epoch 8/30
30/30 [==============================] - 0s 8ms/step - loss: 0.1792 - val_loss: 0.1821
Epoch 9/30
30/30 [==============================] - 0s 7ms/step - loss: 0.1782 - val_loss: 0.1818
Epoch 10/30
30/30 [==============================] - 0s 8ms/step - loss: 0.1773 - val_loss: 0.1819
Epoch 11/30
30/30 

0.17942151427268982

### 768 -> 256

In [ ]:
model_direct_768_to_256= Direct_768_to_256()
batch_size = 64
epochs=50
hist = model_direct_768_to_256.fit(emb_MPnet_train, emb_VGG_av_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(emb_MPnet_valid , emb_VGG_av_valid))
model_direct_768_to_256.evaluate(emb_MPnet_test, emb_VGG_av_test)

Epoch 1/50
30/30 [==============================] - 2s 10ms/step - loss: 0.3314 - val_loss: 0.4462
Epoch 2/50
30/30 [==============================] - 0s 7ms/step - loss: 0.2902 - val_loss: 0.4326
Epoch 3/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2804 - val_loss: 0.4178
Epoch 4/50
30/30 [==============================] - 0s 7ms/step - loss: 0.2739 - val_loss: 0.4012
Epoch 5/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2667 - val_loss: 0.3836
Epoch 6/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2606 - val_loss: 0.3664
Epoch 7/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2562 - val_loss: 0.3529
Epoch 8/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2528 - val_loss: 0.3389
Epoch 9/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2505 - val_loss: 0.3279
Epoch 10/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2463 - val_loss: 0.3177
Epoch 11/50
30/30 

0.2572932839393616

In [ ]:
model_direct_768_to_256= Direct_768_to_256()
batch_size = 64
epochs=50
hist = model_direct_768_to_256.fit(emb_yolo_sent_train, emb_VGG_av_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(emb_yolo_sent_valid , emb_VGG_av_valid))
model_direct_768_to_256.evaluate(emb_yolo_sent_test, emb_VGG_av_test)

Epoch 1/50
30/30 [==============================] - 2s 11ms/step - loss: 0.3440 - val_loss: 0.3836
Epoch 2/50
30/30 [==============================] - 0s 6ms/step - loss: 0.3013 - val_loss: 0.3781
Epoch 3/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2942 - val_loss: 0.3628
Epoch 4/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2901 - val_loss: 0.3576
Epoch 5/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2854 - val_loss: 0.3486
Epoch 6/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2799 - val_loss: 0.3244
Epoch 7/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2767 - val_loss: 0.3232
Epoch 8/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2723 - val_loss: 0.3155
Epoch 9/50
30/30 [==============================] - 0s 5ms/step - loss: 0.2691 - val_loss: 0.3090
Epoch 10/50
30/30 [==============================] - 0s 6ms/step - loss: 0.2664 - val_loss: 0.3013
Epoch 11/50
30/30 

0.2775530219078064

## Auto-encodeur

### 256 -> 768

In [ ]:
model_end_256_to_768, model_features = getAutoEncoder_256_to_768(encodim_dim = 32)
batch_size = 32
epochs=50
hist = model_end_256_to_768.fit(emb_VGG_av_train, emb_yolo_sent_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(emb_VGG_av_valid, emb_yolo_sent_valid))
model_end_256_to_768.evaluate(emb_VGG_av_test, emb_yolo_sent_test)

KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='featurespace/Relu:0', description="created by layer 'featurespace'")
Epoch 1/50
60/60 [==============================] - 5s 14ms/step - loss: 0.2134 - val_loss: 0.2032
Epoch 2/50
60/60 [==============================] - 1s 11ms/step - loss: 0.2006 - val_loss: 0.1958
Epoch 3/50
60/60 [==============================] - 1s 11ms/step - loss: 0.1955 - val_loss: 0.1934
Epoch 4/50
60/60 [==============================] - 1s 11ms/step - loss: 0.1932 - val_loss: 0.1915
Epoch 5/50
60/60 [==============================] - 1s 11ms/step - loss: 0.1914 - val_loss: 0.1910
Epoch 6/50
60/60 [==============================] - 1s 11ms/step - loss: 0.1907 - val_loss: 0.1901
Epoch 7/50
60/60 [==============================] - 1s 11ms/step - loss: 0.1898 - val_loss: 0.1889
Epoch 8/50
60/60 [==============================] - 1s 16ms/step - loss: 0.1891 - val_loss: 0.1901
Epoch 9/50
60/60 [==============================] - 

0.18225443363189697

In [ ]:
len(set(nearest(model_end_256_to_768.predict(emb_VGG_av_test), emb_MPnet_test, Minsk = 1)))

20/20 [==============================] - 0s 2ms/step


18

In [ ]:
model_256_to_768, model_features = getAutoEncoder_256_to_768(encodim_dim = 32)
batch_size = 32
epochs=50
hist = model_256_to_768.fit(emb_VGG_av_train, emb_yolo_sent_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(emb_VGG_av_valid, emb_yolo_sent_valid))
model_256_to_768.evaluate(emb_VGG_av_test, emb_yolo_sent_test)

KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='featurespace/Relu:0', description="created by layer 'featurespace'")
Epoch 1/50
60/60 [==============================] - 5s 15ms/step - loss: 0.2136 - val_loss: 0.2032
Epoch 2/50
60/60 [==============================] - 1s 12ms/step - loss: 0.2005 - val_loss: 0.1961
Epoch 3/50
60/60 [==============================] - 1s 16ms/step - loss: 0.1967 - val_loss: 0.1947
Epoch 4/50
60/60 [==============================] - 1s 16ms/step - loss: 0.1949 - val_loss: 0.1938
Epoch 5/50
60/60 [==============================] - 1s 16ms/step - loss: 0.1938 - val_loss: 0.1931
Epoch 6/50
60/60 [==============================] - 1s 15ms/step - loss: 0.1929 - val_loss: 0.1929
Epoch 7/50
60/60 [==============================] - 1s 11ms/step - loss: 0.1916 - val_loss: 0.1908
Epoch 8/50
60/60 [==============================] - 1s 11ms/step - loss: 0.1906 - val_loss: 0.1902
Epoch 9/50
60/60 [==============================] - 

0.18106761574745178

In [ ]:
model_768_to_256, model_features = getAutoEncoder_768_to_256(encodim_dim = 32)
batch_size = 64
epochs=30
hist = model_768_to_256.fit(emb_MPnet_train, emb_VGG_av_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(emb_MPnet_valid, emb_VGG_av_valid))
model_768_to_256.evaluate(emb_MPnet_test, emb_VGG_av_test)

KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='featurespace/re_lu_63/Relu:0', description="created by layer 'featurespace'")
Epoch 1/30
30/30 [==============================] - 10s 46ms/step - loss: 0.4192 - val_loss: 0.4280
Epoch 2/30
30/30 [==============================] - 1s 29ms/step - loss: 0.3413 - val_loss: 0.3498
Epoch 3/30
30/30 [==============================] - 1s 20ms/step - loss: 0.3137 - val_loss: 0.3218
Epoch 4/30
30/30 [==============================] - 0s 14ms/step - loss: 0.3076 - val_loss: 0.3092
Epoch 5/30
30/30 [==============================] - 0s 17ms/step - loss: 0.3034 - val_loss: 0.3051
Epoch 6/30
30/30 [==============================] - 1s 18ms/step - loss: 0.2982 - val_loss: 0.3003
Epoch 7/30
30/30 [==============================] - 1s 21ms/step - loss: 0.2959 - val_loss: 0.2987
Epoch 8/30
30/30 [==============================] - 0s 12ms/step - loss: 0.2901 - val_loss: 0.2948
Epoch 9/30
30/30 [========================

0.26519012451171875

In [ ]:
model_768_to_256, model_features = getAutoEncoder_768_to_256(encodim_dim = 32)
batch_size = 256
epochs=50
hist = model_768_to_256.fit(emb_yolo_sent_train, emb_VGG_av_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(emb_yolo_sent_valid, emb_VGG_av_valid))

KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='featurespace/re_lu_70/Relu:0', description="created by layer 'featurespace'")
Epoch 1/50
8/8 [==============================] - 5s 50ms/step - loss: 0.4727 - val_loss: 0.4653
Epoch 2/50
8/8 [==============================] - 0s 17ms/step - loss: 0.3549 - val_loss: 0.4355
Epoch 3/50
8/8 [==============================] - 0s 17ms/step - loss: 0.3237 - val_loss: 0.3980
Epoch 4/50
8/8 [==============================] - 0s 17ms/step - loss: 0.3121 - val_loss: 0.3667
Epoch 5/50
8/8 [==============================] - 0s 16ms/step - loss: 0.3074 - val_loss: 0.3462
Epoch 6/50
8/8 [==============================] - 0s 16ms/step - loss: 0.3047 - val_loss: 0.3335
Epoch 7/50
8/8 [==============================] - 0s 17ms/step - loss: 0.3024 - val_loss: 0.3237
Epoch 8/50
8/8 [==============================] - 0s 17ms/step - loss: 0.3022 - val_loss: 0.3238
Epoch 9/50
8/8 [==============================] - 0s 15ms/s

# Model pour classif

In [ ]:
np.shape(emb_VGG_av_train), np.shape(emb_yolo_sent_train), np.shape(emb_MPnet_train)

((1909, 256), (1909, 768), (1909, 768))

In [ ]:
import tensorflow as tf

In [ ]:
train_concat = np.concatenate((emb_VGG_av_train, emb_yolo_sent_train,emb_MPnet_train), axis = 1)
test_concat = np.concatenate((emb_VGG_av_test, emb_yolo_sent_test,emb_MPnet_test), axis = 1)
valid_concat = np.concatenate((emb_VGG_av_valid, emb_yolo_sent_valid,emb_MPnet_valid), axis = 1)

In [ ]:
def ConcatSplit(dropout = True):
  x_in = Input((256 + 768 + 768))
  x = x_in
  x_audio, x_image, x_text = tf.split(x, [256, 768, 768], 1)
  layers_1=[1024, 512, 256, 128]

  for x in [x_audio, x_image, x_text]:
    x = BatchNormalization()(x)
    for layer in layers_1:
      x = Dense(layer, activation = 'ReLU')(x)
      if dropout:
        x = Dropout(0.8)(x)

  x = tf.concat([x_audio, x_image, x_text], 1)
  x = BatchNormalization()(x)

  layers_2=[256, 128]
  for layer in layers_2:
    x = Dense(layer, activation = 'ReLU')(x)
    if dropout:
      x = Dropout(0.8)(x)
  x_out = Dense(15, activation = 'sigmoid')(x)
  model = Model(x_in, x_out)
  model.compile(loss = 'categorical_crossentropy', metrics = 'accuracy', optimizer = Adam())
  return model

model_split =  ConcatSplit()
#model.summary()

In [ ]:
batch_size = 128
epochs=40
hist = model_split.fit(train_concat, np.array(pd.get_dummies(df_train['label'])),
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(valid_concat,  np.array(pd.get_dummies(df_valid['label']))))

Epoch 1/40
15/15 [==============================] - 0s 10ms/step - loss: 0.9573 - accuracy: 0.6611 - val_loss: 2.0916 - val_accuracy: 0.4382
Epoch 2/40
15/15 [==============================] - 0s 7ms/step - loss: 0.9425 - accuracy: 0.6668 - val_loss: 2.0845 - val_accuracy: 0.4444
Epoch 3/40
15/15 [==============================] - 0s 8ms/step - loss: 0.9149 - accuracy: 0.6726 - val_loss: 2.1214 - val_accuracy: 0.4476
Epoch 4/40
15/15 [==============================] - 0s 7ms/step - loss: 0.9205 - accuracy: 0.6731 - val_loss: 2.1222 - val_accuracy: 0.4397
Epoch 5/40
15/15 [==============================] - 0s 8ms/step - loss: 0.9251 - accuracy: 0.6752 - val_loss: 2.1193 - val_accuracy: 0.4382
Epoch 6/40
15/15 [==============================] - 0s 8ms/step - loss: 0.9203 - accuracy: 0.6700 - val_loss: 2.1214 - val_accuracy: 0.4429
Epoch 7/40
15/15 [==============================] - 0s 8ms/step - loss: 0.9125 - accuracy: 0.6799 - val_loss: 2.1288 - val_accuracy: 0.4460
Epoch 8/40
15/15 [=

In [ ]:
pred = model_split.predict(train_concat)
np.mean(np.argmax(np.array(pd.get_dummies(df_train['label'])), axis = 1) == np.argmax(pred, axis = 1))

60/60 [==============================] - 0s 2ms/step


0.9533787323205867

In [ ]:
pred = model_split.predict(valid_concat)
np.mean(np.argmax(np.array(pd.get_dummies(df_valid['label'])), axis = 1) == np.argmax(pred, axis = 1))

20/20 [==============================] - 0s 2ms/step


0.4460093896713615

In [ ]:
pred = model_split.predict(test_concat)
np.mean(np.argmax(np.array(pd.get_dummies(df_test['label'])), axis = 1) == np.argmax(pred, axis = 1))

20/20 [==============================] - 0s 2ms/step


0.384375

In [ ]:
show_conf_matrix(np.argmax(np.array(pd.get_dummies(df_test['label'])), axis = 1), np.argmax(pred, axis = 1))

In [ ]:
predicted_15 = np.zeros((len(df_test),15))
for count, elem in enumerate(predicted):
  predicted_15[count, elem] = 1

In [ ]:
np.array(pd.get_dummies(df_test['label']))

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
np.save('best_til_now.npy', pred)

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[10, 50, 150], 'gamma': ['scale']}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit( train_concat,df_train['label'])
pd.DataFrame(clf.cv_results_)

KeyboardInterrupt: ignored

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='scale', C=100, kernel = 'rbf', class_weight = 'balanced'))
clf.fit( train_concat,df_train['label'])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=100, class_weight='balanced'))])

In [ ]:
tfsvmpred = clf.predict(train_concat)
accuracy_score(df_train['label'],tfsvmpred)

0.9958093242535359

In [ ]:
tfsvmpred = clf.predict(valid_concat)
accuracy_score(df_valid['label'],tfsvmpred)

0.41001564945226915

In [ ]:
show_conf_matrix(df_valid['label'],tfsvmpred)

In [ ]:
tfsvmpred = clf.predict(test_concat)
accuracy_score(df_test['label'],tfsvmpred)

0.4078125

In [ ]:
show_conf_matrix(df_test['label'],tfsvmpred)